In [ ]:
from database.database import postgersql,mongodb
from config.postgres  import  table_frs_option
import numpy as np
import pandas as pd

In [ ]:
sql = postgersql()
mongo = mongodb()

In [1]:
import keras

ImportError: No module named keras

In [ ]:
colname = [
             'id',    
             'trade_id',
             'currency_pair',
             'sell_currency',
             'buy_currency',
             'sell_amount',
             'trade_date',
             'determined_date',
             'delivery_date',
             'trp',
             'rate'
                ]
data = sql.select(table_frs_option,colname,None)
data = pd.DataFrame(data)

In [ ]:
data

In [ ]:
currency_pairs = list(set(map(lambda x:x['currency_pair'],data)))

In [ ]:
from database.mongodb import  RateExchange
from database.mongodb import  BankRate

In [ ]:
currency_dict = {}
for currency_pair in currency_pairs:
            RE = RateExchange(currency_pair).getMax()
            if RE is not None and RE !=[]:
               currency_dict[currency_pair] = RE[0]['Close']

In [ ]:
currency_dict

In [ ]:
##获取每天的汇率
spot = mongo.select('kline',{'type':'5','code':'AUDCNY'})

spot = pd.DataFrame(spot)
spot['Close'] = spot['Close']/spot['PriceWeight']
spot['Time'] = spot['Time'].astype(np.datetime64).dt.strftime('%Y-%m-%d')


In [ ]:
from help.help import timedelta

In [ ]:
def datafill(spot):
    """填充无交易日期的汇率"""
    def dateseris(mindate,maxdate):
        t = 0
        date = timedelta(mindate,1)
        series = []
        while maxdate>=date:
            series.append(date)

            date = timedelta(date,1)

        return series

    spot = pd.merge(pd.DataFrame(dateseris(spot['Time'].min(),spot['Time'].max()),columns=['Time']),spot[['Time','Close']],on=['Time'],how='left' )
    spot = spot.sort_values('Time')
    for i in range(spot.shape[0]):
        if spot['Close'].values[i].__str__()=='nan':
            spot['Close'].values[i] = spot['Close'].values[i-1]
    return spot

def lagdata(spot,lags=[30,60,90,120,150,180,210,240,270,300,330,360]):
    spot = datafill(spot)
    for lag in lags:
        spot['Close_%d'%lag] = np.repeat(None,lag).tolist() + spot['Close'].values[:-lag].tolist()
    spot =  spot.dropna()
    
    for lag in lags:
        spot['Close_%d_rate'%lag] = (spot['Close'] - spot['Close_%d'%lag])/spot['Close_%d'%lag]
    
    return spot    

In [ ]:
spot = datafill(spot)
spot = lagdata(spot,[30,60,90,120])

In [ ]:
spot

In [ ]:
def getkline(code,date):
    """
    code：汇率对
    date：日期
    获取指定日期指定汇率对的汇率
    """
    date = date+" 00:00:00"
    reslut =  mongo.select('kline',{'type':'5','code':code,'Time':date})
    if reslut!=[]:
        return reslut[0].get('High') /1.0 / reslut[0].get('PriceWeight')
    else:
        return None

    
    

In [ ]:
##厘定日汇率
data['determined_date_rate'] = data[['currency_pair','determined_date']].apply(func = lambda x:getkline(x[0],str(x[1])),axis=1)

In [ ]:
data

In [ ]:
##蒙特卡洛模拟未来十二个月的每个月汇率情况
def simulation(S,rate,times):
    pred = []##模拟值
    for i in range(times):
        r = np.random.choice(rate)
        if pred ==[]:
            
            
            pred.append(S*(1+r))
        else:
            pred.append(pred[-1]*(1+r))
    return pred
simulation(5.1011,spot['Close_30_rate'].values,10)

In [ ]:
那个太长了，可能是逻辑导致的，以获取牌价为例目前的方式是：
1.先group by 银行、币种，获得他们的最大发布时间（也就是最近更新的时间）；
	得到了一大堆数据。
2.再用 银行、币种、时间三个条件去匹配，然后这一大堆数据之前是or关系，就比如：
{$match: {$or: 
	[
		{bank : 'BOC', currency : 'USD', publishDateTime : '2016-09-23 12:12:12'},
		{bank : 'BOC', currency : 'EUR', publishDateTime : '2016-09-23 10:57:00'},
		…
	]
}
 


function getbankspot(bank,limitnum=1000){
        var result= {};
        var temp = db.getSiblingDB('fes')[bank].find().sort({'DataDate':-1}).limit(limitnum);
        temp.forEach(
            function(x){
                if (result.hasOwnProperty(x.banktype)){
                    if (result[x.banktype].hasOwnProperty(x.currency)){
                        if (result[x.banktype][x.currency]['publishDateTime']< x.publishDateTime){
                            result[x.banktype][x.currency] =x;
                        }
                    }
                    else{
                            result[x.banktype][x.currency]=x;
                        }
                }
                else{
                    result[x.banktype] ={};
                    result[x.banktype][x.currency] =x;
                }
                
            }
            
        );
        var data = []
        for(banktype in result){
            for(currency in result[banktype]){
                data.push(result[banktype][currency])
            }
        }
        return data;  
      
    }
 


In [ ]:
那个太长了，可能是逻辑导致的，以获取牌价为例目前的方式是：
1.先group by 银行、币种，获得他们的最大发布时间（也就是最近更新的时间）；
	得到了一大堆数据。
2.再用 银行、币种、时间三个条件去匹配，然后这一大堆数据之前是or关系，就比如：
{$match: {$or: 
	[
		{bank : 'BOC', currency : 'USD', publishDateTime : '2016-09-23 12:12:12'},
		{bank : 'BOC', currency : 'EUR', publishDateTime : '2016-09-23 10:57:00'},
		…
	]
}

function getspot(bank){
        var currencys = db.getSiblingDB('fes')[bank].distinct('currency');
        var data =[];
        types =  ['BOC','ABC','ICBC','CCB'];
        
        for(index in currencys){
            
            for(type_ in types){
               temp = db.getSiblingDB('fes')[bank].find({'currency':currencys[index],'banktype':types[type_]});
               if(temp.hasNext()){
                       data.push(temp .next());
                    }
                                
            }
        }
        return data;}
 
        var result= {};
        var db.getSiblingDB('fes')[bank].distinct('currency')
        for 
        var t = new Date();
        var temp = db.getSiblingDB('fes')[bank].find({'date':{'$gte':date}})
        
        temp.forEach(
            function(x){
                if (result.hasOwnProperty(x.banktype)){
                    if (result[x.banktype].hasOwnProperty(x.currency)){
                        if (result[x.banktype][x.currency]['publishDate']< x.publishDate){
                            result[x.banktype][x.currency] =x;
                        }
                        
                    }
                    else{
                            result[x.banktype][x.currency]=x;
                        }
                }
                else{
                    result[x.banktype] ={};
                    result[x.banktype][x.currency] =x;
                }
                
            }
            
        );
       print(new Date() -t); 
        
        return result;  
      
    }
 


In [ ]:
var temp = db.getSiblingDB('fes')['bank'].find()
temp.forEach(
            function(x){
                if(result.hasOwnProperty(x.banktype)){
                    if result[x.banktype].hasOwnProperty(x.currency){
                        if result[x.banktype][x.currency]['publishDate']< x.publishDate{
                            result[x.banktype][x.currency]['publishDate'] = x.publishDate;
                            result[x.banktype][x.currency]['spotBuy'] = x.spotBuy;
                            result[x.banktype][x.currency]['cashSell'] = x.cashSell;
                            result[x.banktype][x.currency]['cashBuy'] = x.cashBuy;
                            result[x.banktype][x.currency]['spotSell'] = x.spotSell;
                            
                             }
                        
                    }
                    else{
                            result[x.banktype][x.currency]=x;
                        }
                }
        }
    )
        
                else{
                    result[x.banktype] ={};
                    result[x.banktype][x.currency] =x;
                }
                
            }
            
        );

In [ ]:
db.bank.ensureIndex({ "banktype" : 1,"currency" : 1, "date" : -1, "publishDate" : -1},{"unique" : true})
db.kline_2016_09_25.ensureIndex({"code" : 1,"type" : 1,"Time" : -1})

In [ ]:
import t